## Tutorial 1 smolagent
https://huggingface.co/docs/smolagents/guided_tour?Pick+a+LLM=HF+Inference+API

NOTA: Recomendable ejecutarlo en un entorno virtual de python

In [ ]:
!pip install smolagents smolagents[litellm] python-dotenv

In [ ]:
!python.exe -m pip install --upgrade pip

In [3]:
model_id = "meta-llama/Llama-3.3-70B-Instruct" 
# model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

### Cargar la clave de hugging face desde el fichero .env

In [ ]:
from dotenv import load_dotenv

load_dotenv()

 ### Crear el modelo con LiteLLM (los modelos creado con HF Inference API parece ser que consumen más tokens)

In [5]:
from smolagents import LiteLLMModel

model = LiteLLMModel(model_id='huggingface/'+model_id)

### Crear el agente

In [26]:
prompt_templates = agent.prompt_templates
agent.prompt_templates['system_prompt'] += "\nWhen you generate a final answer, you must stop immediately."

In [27]:
from smolagents import CodeAgent

agent = CodeAgent(tools=[], model=model, add_base_tools=True)
set_system_prompt(agent)

### Ejemplo de uso del agente

In [ ]:
result = agent.run(
    "Could you give me the 118th number in the Fibonacci sequence?",
)

print(result)

El resultado anterior algunas veces sale mal.

UNA SOLUCION:

Si se define la función de fibonacci como una tool y ya da el resultado correcto:

NOTA: Cuando se usa @tool, para que no salga un error, dentro de def es obligatorio el bloque siguiente:

    """
    Calcula el enésimo número de Fibonacci.

    Args:
        n: Posición del número de fibonacci a calcular.
    """

En la documentación lo indican así:

The function needs:

A clear name.
The name should be descriptive enough of what this tool does to help the LLM brain powering the agent. Since this tool returns the model with the most downloads for a task, let’s name it model_download_tool.
Type hints on both inputs and output.

A description, that includes an ‘Args:’ part where each argument is described (without a type indication this time, it will be pulled from the type hint). Same as for the tool name, this description is an instruction manual for the LLM powering you agent, so do not neglect it. All these elements will be automatically baked into the agent’s system prompt upon initialization: so strive to make them as clear as possible!

In [33]:
from smolagents import tool

@tool
def fibonacci(n: int) -> int:
    """
    Calcula el enésimo número de Fibonacci.

    Args:
        n: Posición del número de fibonacci a calcular.
    """

    a, b = 0, 1
    for _ in range(n - 1):
        a, b = b, a + b
    return a

# Configurar el agente con la tool de Fibonacci que acabamos de definir
agent = CodeAgent(tools=[fibonacci], model=model, add_base_tools=True)

NOTA: Se puede apreciar que hay un error interno porque en el step 2 ya se detecta final_answer, pero sigue en ejecución consumiendo tokens hasta el límite de pasos establecido.

El siguiente código modifica el system_prompt para corregir este error:

In [34]:
prompt_templates = agent.prompt_templates
agent.prompt_templates['system_prompt'] += "\nWhen you generate a final answer, you must stop immediately."

In [ ]:
# Ejecutar la consulta al agente
response = agent.run("What is the 118th Fibonacci number?")

print("Response:", response)

### Otra forma de definir la tool pero usando una class

### Crear la class

In [31]:
from smolagents.tools import Tool

# Definir la clase Fibonacci heredando de Tool
class FibonacciTool(Tool):
    name = "fibonacci"  # Agregamos el atributo name
    description = "Calculates the nth Fibonacci number."
    inputs = {
        "n": {
            "type": "integer",
            "description": "The position in the Fibonacci sequence to calculate"
        }
    }
    output_type = "integer"  # Añadimos el tipo de salida
     
    def forward(self, n: int) -> int:
        """Calcula el enésimo número de Fibonacci."""
        a, b = 0, 1
        for _ in range(n - 1):
            a, b = b, a + b
        return a

# Crear una instancia de la herramienta
fibonacci_tool = FibonacciTool()

### Crear el agente

In [32]:
agent = CodeAgent(tools=[fibonacci_tool], model=model, add_base_tools=True)

NOTA: Se puede apreciar que hay un error interno porque en el step 2 ya se detecta final_answer, pero sigue en ejecución consumiendo tokens hasta el límite de pasos establecido.

En este caso, el siguiente código modifica el system_prompt pero no consigue corregir este error:

In [33]:
prompt_templates = agent.prompt_templates
agent.prompt_templates['system_prompt'] += "\nWhen you generate a final answer, you must stop immediately."

In [ ]:
# Ejecutar la consulta al agente
response = agent.run("What is the 118th Fibonacci number?")

print("Response:", response)

### Inspecting an agent run

Here are a few useful attributes to inspect what happened after a run:

agent.logs

Stores the fine-grained logs of the agent. At every step of the agent’s run, everything gets stored in a dictionary that then is appended to agent.logs.

Running agent.

write_memory_to_messages() writes the agent’s memory as list of chat messages for the Model to view. This method goes over each step of the log and only stores what it’s interested in as a message: for instance, it will save the system prompt and task in separate messages, then for each step it will store the LLM output as a message, and the tool call output as another message. Use this if you want a higher-level view of what has happened - but not every log will be transcripted by this method.

In [ ]:
import pprint

# agent.logs    # deprecated, use agent.memory.steps

pprint.pprint(agent.memory.steps)print("Response:", response)